## Content-based filtering

Import the needs for this to work

In [1]:
import pandas as pd
import pickle
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import NorwegianStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
import operator

In [2]:
stemmer = NorwegianStemmer() 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vy111\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
class NLP():
    def __init__(self, data):
        self.data = data
        # self.nlp = spacy.load('nb_core_news_sm')
        self.vectors = None


    def preprosses_df(self):
        self.info("Removing None and duplicates in data")
        self.data = self.data.drop_duplicates(["title"])
        self.data = self.data.dropna(subset=('title',))
        self.data = self.data.reset_index()
  
    def get_titles(self):
        self.info("Getting the titles")
        titles = self.data["title"]
        titles = [title for title in titles]
        return titles

    def remove_noice(self, titles):
        self.info("Removing the noice in the titles")
        PUNCT = string.punctuation+"«»"
        for index, title in enumerate(titles):
            titles[index] = title.translate(str.maketrans('', '', PUNCT))
        return titles

    def normalize(self, titles):
        self.info("Stemming the words")
        for index, title in enumerate(titles):
            titles[index] = [stemmer.stem(word) for word in title]
        return titles

    def remove_stopwords(self, titles):
        self.info("Removing the stopwords in the titles")

        for index, title in enumerate(titles):
            titles[index] = [word for word in title if not word in stopwords.words("norwegian")]
        return titles

    
    def tokenize(self, titles):
        self.info("Tokenizing and lowering the word in titles")
        for index in range(len(titles)):
            titles[index] =  nltk.word_tokenize(titles[index].lower())
        return titles

    def do_magic(self):
        self.info("Init prosses")
        self.preprosses_df()
        titles = self.get_titles()

        titles = self.remove_noice(titles)

        titles = self.tokenize(titles)
        
        titles = self.remove_stopwords(titles)
        
        titles = self.normalize(titles)

        self.data["new_title"] = titles
        self.TFIDF()

    def TFIDF(self):
        self.info("Vectorizing the corpus with TFIDF")
        vectorizer = TfidfVectorizer()
        corpus = [" ".join(sentenc) for sentenc in self.data["new_title"].tolist()]
        self.vectors = vectorizer.fit_transform(corpus)
        # denselist = vectors.todense().tolist()

    def compare(self, title, k_number=10):
        #self.info("Comparing title with all other titles")
        #self.info(f"Search: [{title}]")
        q_index = self.data[self.data["title"]==title].index
        if len(q_index)<=0:
            return None

        similarity = cosine_similarity(self.vectors)[q_index].tolist()[0]
        # for index,value in enumerate(similarity):
        #     self.data.iloc[index]["similarity"] = value
        # print(similarity)
        # index, value = max(enumerate(similarity), key=operator.itemgetter(1))
        # print("------------------",index, value)
        # print(self.data.iloc[index])
        # self.data = self.data.sort_values(by=['index'])
        self.data["similarity"] = similarity
        results = self.data.sort_values(by=['similarity'], ascending=False)
        return results[:k_number]

    def info(self, text):
        line = "="*(75-len(text))
        print(f"{line}=> {text} +  {len(self.data)}")

In [4]:
# if __name__ == "__main__":
#     path = "data/pickeledDataset"
#     df = pickle.load(open(path, "rb"))
#     nlp = NLP(df)
#     nlp.do_magic()

#     i = random.randint(0, len(nlp.data))
#     query = nlp.data["title"][i]
   
#     results = nlp.compare(query, k_number = 15)
#     cols = ["similarity","title", "url"]

### Unpickle the pickle
The data for the system

In [5]:
path = "data/pickeledDataset"
df = pickle.load(open(path, "rb"))

### Preprosses the data


In [6]:
nlp = NLP(df)
nlp.do_magic()

================================================================> Init prosses +  680355
========================================> Removing None and duplicates in data +  680355
==========================================================> Getting the titles +  18958
============================================> Removing the noice in the titles +  18958
==================================> Tokenizing and lowering the word in titles +  18958
========================================> Removing the stopwords in the titles +  18958
==========================================================> Stemming the words +  18958
===========================================> Vectorizing the corpus with TFIDF +  18958


### Find a random user

It will find a user and retrive all the data from this userId.

In [7]:
uniqueUsers = pd.unique(df['userId'])
print(len(uniqueUsers))

1000


In [8]:
index = random.randint(0, len(nlp.data))
user = df["userId"][index]
userData = df.loc[lambda df: df['userId'] == user]

It will then filter out the data where the activtime is less then 44. This will be the users relevant dataset, and it will be used to measure recall and ARHR.

In [9]:
relevantUserData = userData.loc[userData['activeTime'] > 44]

### Make a search with a existing title form the data set

Further, it will then choose a random title from the relevant dataset from the random userId.

In [10]:
#Uses a random number
indexUser = random.randint(0, len(userData))
title = userData.sample()["title"].iat[0]
print(title)

Eggen om Bendtner: - Skal noen få skikk på ham, er det «guttene mine» på Lerkendal


### Compare

It will compare various titles by using cosine-similarity. An list of the most top 15 articles will show.

In [11]:
top_k_results = 15
res = nlp.compare(title, k_number=top_k_results)
res

,index,eventId,category,activeTime,title,url,userId,publishtime,time,documentId,new_title,similarity
2723,51456,1725560276,100sport|fotball,46.000000,Eggen om Bendtner: - Skal noen få skikk på ham...,http://adressa.no/100sport/fotball/eggen-om-be...,cx:13497671501361832862782:gx2qsg854oeh,2017-03-07T09:07:05.000Z,1489157677,966e5991bde1adf3514abe45e0350e835e663697,"[egg, bendtn, få, skikk, ham, gutt, lerkendal]",1.000000
14128,425994,1831977896,None,14.000000,Direkte: Bendtner møter pressen på Lerkendal,http://adressa.no/video/article14448597.ece,cx:2h6xqt8cgq8c43k4ofnzmkxxsk:1wl24qruz4cvw,2017-03-15T11:30:25.000Z,1489584222,6997b7853dcd1469b2f5a0ee76f225d15b03634a,"[direkt, bendtn, møt, press, lerkendal]",0.384070
4878,101838,1513805065,100sport|fotball,44.359582,Her får Bendtner sin første leksjon av mester ...,http://adressa.no/100sport/fotball/her-far-ben...,cx:1383860356863849037094:3gopbqslccea4,2017-03-23T14:38:15.000Z,1490818798,e6b13b7d0088af3818cf750a5e5564901cd05967,"[får, bendtn, først, leksjon, mest, egg]",0.341114
12786,364167,277501262,nyheter|trondheim,185.000000,Nå skal denne gata få et skikkelig ansiktsløft,http://adressa.no/nyheter/trondheim/2016/11/28...,cx:hst6hn5p7iv3i23j:2fnwe999efaax,2016-11-28T21:15:22.000Z,1490185611,9c54ac864a1bbb7d6b602e50a28a78b12930625c,"[gat, få, skikk, ansiktsløft]",0.313409
12676,358508,1336521630,nyheter|sortrondelag,27.000000,- Det var et skikkelig smell,http://adressa.no/nyheter/sortrondelag/2015/12...,cx:1kfmbn5m74t221malc73lr3w32:2lb8hkquy7t23,2015-12-27T14:46:51.000Z,1484065862,0fece9aec60a4604a72b47c3e843119071827e59,"[skikk, smell]",0.304917
3182,61680,1712503038,100sport|andreidretter,1.000000,Eggen gleder seg,http://adressa.no/norge/okt-pagang-av-barn-som...,cx:il4sn6l573yshivi:10w3zs8jeih3k,2000-11-11T10:57:00.000Z,1489779983,f34fcf2af732748c9d19f4a9dc585fd9997c1147,"[egg, gled]",0.300953
11305,303989,1898509524,nyheter|trondheim,44.359582,- Det er skikkelig tragisk,http://adressa.no/nyheter/trondheim/2016/12/28...,cx:1kfmbn5m74t221malc73lr3w32:2lb8hkquy7t23,2016-12-28T08:23:03.000Z,1483941034,b09a1b0da50865b3ab82024d64852f44d214041c,"[skikk, tragisk]",0.278876
8687,216671,1462815619,nyheter|trondheim,189.000000,«Få skikk på eldreomsorgen før superbusser og ...,http://adressa.no/nyheter/trondheim/2017/02/24...,cx:2j5iepyewbe683niz1zfmuu4vc:1la1o1zm5h4pe,2017-02-24T19:08:11.000Z,1487977754,ed78a5af96a0292b9f6a6475ba850b6af5abf3bd,"[få, skikk, eldreomsorg, superbuss, andr, pres...",0.259657
14735,456687,517943970,None,44.359582,- Vi vil at guttene og deres familie skal få k...,http://adressa.no/nyheter/moreromsdal/article1...,cx:ilci32s5jzhxgdwf:2qur4fhg4shzx,None,1484166291,9a74fbbd72145ce862eedcad1ac8963cda5816a9,"[gutt, famili, få, komm, tilbak, sunndal]",0.258074
12040,334329,1652890079,pluss|sport,112.000000,Han styrer lyden av Lerkendal,http://adressa.no/pluss/sport/2015/11/04/han-s...,cx:ie41rzokxgss9y1d:3l7vyyb4y74b,2015-11-04T09:23:44.000Z,1490446161,7bc561b38ea6fc09f71fe681b815b7a9452a7f5f,"[styr, lyd, lerkendal]",0.251140


### Better view

Here we can see the top 15 articles with their corresponding similarity.

In [12]:
cols = ["similarity","title", "activeTime"]
res[cols]

,similarity,title,activeTime
2723,1.000000,Eggen om Bendtner: - Skal noen få skikk på ham...,46.000000
14128,0.384070,Direkte: Bendtner møter pressen på Lerkendal,14.000000
4878,0.341114,Her får Bendtner sin første leksjon av mester ...,44.359582
12786,0.313409,Nå skal denne gata få et skikkelig ansiktsløft,185.000000
12676,0.304917,- Det var et skikkelig smell,27.000000
3182,0.300953,Eggen gleder seg,1.000000
11305,0.278876,- Det er skikkelig tragisk,44.359582
8687,0.259657,«Få skikk på eldreomsorgen før superbusser og ...,189.000000
14735,0.258074,- Vi vil at guttene og deres familie skal få k...,44.359582
12040,0.251140,Han styrer lyden av Lerkendal,112.000000


### Measurements

#### Recall

In [13]:
# Recall
retLis = res[cols].reset_index()
getRel = userData.merge(retLis, how='inner', on='title')
recall = len(getRel) / len(relevantUserData)
print("Recall:", recall)

Recall: 0.0013157894736842105


#### Precision

In [14]:
# Precision
precision = len(getRel) / len(retLis)
print("Precision:", precision)

Precision: 0.06666666666666667


#### ARHR

In [15]:
retLis["index2"] = np.arange(1,len(res[cols])+1)

In [30]:
common = retLis.merge(userData, how='inner', on='title')
common

,index,similarity,title,activeTime_x,index2,eventId,category,activeTime_y,url,userId,publishtime,time,documentId
0,2160,1.000000,Mat til kreftsyke ble spist opp i 30-årslag:,57.0,1,554397485,nyheter|trondheim,44.359582,http://adressa.no/nyheter/trondheim/2017/01/16...,cx:37qlonca27pkr2pj5ez4e997l1:3spqe7w2c9ur4,2017-01-16T20:35:08.000Z,1484632398,fec01e53f1c622b11ce8b842010ee33f097fd300
1,8925,0.259579,Kreftsyke barn har tegnet seg inn på det norsk...,1.0,5,1924134267,100sport|vintersport,44.359582,http://adressa.no/100sport/vintersport/kreftsy...,cx:37qlonca27pkr2pj5ez4e997l1:3spqe7w2c9ur4,2017-02-23T16:19:12.000Z,1487872778,bcbad89cbb12ed3cc8cd4441a02ab45d2c5207b6


In [17]:
# ARHR
ARHR = sum(1/common["index2"])/len(common)
print("ARHR:", ARHR)

ARHR: 1.0


### Now let's do it for all users

In [38]:
precisions = []
recalls = []
arhrs = []


for idx, user in enumerate(pd.unique(df['userId'])):
    if idx % 50 == 0:
        print(idx, "/", len(pd.unique(df['userId'])))
    index = random.randint(0, len(nlp.data))
    userData = df.loc[lambda df: df['userId'] == user]
    relevantUserData = userData.loc[userData['activeTime'] > 44]
    
    #Uses a random number
    indexUser = random.randint(0, len(userData))
    title = userData.sample()["title"].iat[0]

    top_k_results = 15
    res = nlp.compare(title, k_number=top_k_results)
    if res.empty:
        print("skip")
        print(title)
        continue

    cols = ["similarity","title", "activeTime"]

    # Recall
    retLis = res[cols].reset_index()
    getRel = userData.merge(retLis, how='inner', on='title')
    recall = len(getRel) / len(relevantUserData)
    recalls.append(recall)

    # Precision
    precision = len(getRel) / len(retLis)
    precisions.append(precision)

    retLis["index2"] = np.arange(1,len(res[cols])+1)

    common = retLis.merge(userData, how='inner', on='title')

    # ARHR
    ARHR = sum(1/common["index2"])/len(common)
    arhrs.append(ARHR)

print(arhrs)

0 / 1000
50 / 1000
100 / 1000
150 / 1000
200 / 1000
250 / 1000
300 / 1000
350 / 1000
400 / 1000
450 / 1000
500 / 1000
550 / 1000
600 / 1000
650 / 1000
700 / 1000
750 / 1000
800 / 1000
850 / 1000
900 / 1000
950 / 1000
[0.75, 1.0, 0.5833333333333334, 0.5357142857142857, 0.5555555555555556, 1.0, 1.0, 0.5833333333333334, 1.0, 1.0, 1.0, 0.2381410256410256, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 0.2891219891219891, 0.5555555555555556, 0.6666666666666666, 1.0, 1.0, 0.38244047619047616, 1.0, 0.5625, 0.625, 0.4222222222222222, 0.5454545454545454, 0.3849206349206349, 1.0, 0.5357142857142857, 0.3361950549450549, 0.5357142857142857, 1.0, 0.5357142857142857, 1.0, 1.0, 1.0, 0.4861111111111111, 1.0, 0.6666666666666666, 1.0, 0.625, 0.34958791208791207, 1.0, 1.0, 1.0, 0.5416666666666666, 1.0, 0.48333333333333334, 0.5625, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 0.49206349206349204, 0.425, 1.0, 1.0, 0.396969696969697, 1.0, 0.75, 0.5416666666666666, 1.0, 

In [39]:
sum(arhrs) / len(arhrs)

0.7545433761628193

snitt: 0.7516660735208348